In [ ]:
import telegram
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import pandas as pd
import re

# Функция для очистки текста
def clean_text(text):
    text = text.strip().lower()
    text = re.sub(r'[^\w\s.,!?:;]', '', text)
    return text

# Загрузка данных
def load_comments():
    try:
        df = pd.read_csv('Main_комментарии.csv', on_bad_lines='skip', engine='python', encoding='utf-8')
        comment_dict = {}

        if 'comment' in df.columns and 'rating' in df.columns:
            for _, row in df.iterrows():
                comment = str(row['comment']).strip()
                rating = int(row['rating'])
                normalized = clean_text(comment)
                comment_dict[normalized] = rating
        else:
            with open('разметка комментариев 2.csv', 'r', encoding='utf-8') as f:
                lines = [line.strip() for line in f if line.strip()]
            i = 0
            while i < len(lines):
                parts = lines[i].split(',')
                try:
                    rating = int(parts[1])
                    comment = ','.join(parts[2:]).strip()
                    normalized = clean_text(comment)
                    comment_dict[normalized] = rating
                    i += 1
                except:
                    i += 1
        return comment_dict
    except Exception as e:
        print("Ошибка загрузки данных:", e)
        return {}

# Определение тональности
def get_sentiment(rating):
    if rating == 5:
        return "🟢 Позитивный"
    elif rating == 4:
        return "🟠 Нейтральный"
    else:
        return "🔴 Негативный"

# Обработчики
async def start(update: telegram.Update, context: telegram.ext.CallbackContext):
    await update.message.reply_text("Привет! Напиши комментарий, и я скажу, позитивный он или нет.")

async def handle_message(update: telegram.Update, context: telegram.ext.CallbackContext):
    user_input = update.message.text.strip()
    cleaned_input = clean_text(user_input)

    rating = comment_dict.get(cleaned_input)

    if rating is None:
        await update.message.reply_text("Комментарий не найден")
    else:
        sentiment = get_sentiment(rating)
        await update.message.reply_text(f"Рейтинг: {rating}\nТональность: {sentiment}")

# Загрузка комментариев
comment_dict = load_comments()
print("Загружено комментариев:", len(comment_dict))

# Создаем приложение
app = ApplicationBuilder().token("7917572552:AAG4PrRjBV578gPy0WupsvsX_y0WL7C7z-Y").build()

# Добавляем обработчики
app.add_handler(CommandHandler("start", start))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

# Для Jupyter Notebook
import asyncio
import nest_asyncio

nest_asyncio.apply()  # Применяем патч для async

async def run_bot():
    await app.initialize()
    await app.start()
    await app.updater.start_polling()
    print("Бот запущен...")

# Запуск бота в текущем цикле
asyncio.get_event_loop().create_task(run_bot())

Загружено комментариев: 1682


<Task pending name='Task-1' coro=<run_bot() running at C:\Users\2k\AppData\Local\Temp\ipykernel_14100\1133990886.py:85>>

Бот запущен...


Exception happened while polling for updates.
Traceback (most recent call last):
  File "C:\Users\2k\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\telegram\ext\_utils\networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\2k\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\telegram\ext\_utils\networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\tasks.py", line 277, in __step
    result = c

Ссылка на бота: t.me/intens0_bot